In [2]:
import numpy as np
import pandas as pd
from sklearn import linear_model
import matplotlib.pyplot as plt
from sqlalchemy import create_engine, text

import warnings
warnings.filterwarnings('ignore')

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'medicalcosts'

In [18]:
#df = pd.read_csv("https://github.com/bot13956/ML_Model_for_Predicting_Ships_crew_Size/blob/master/cruise_ship_info.csv")

df = pd.read_csv("https://github.com/bot13956/ML_Model_for_Predicting_Ships_Crew_Size/blob/master/cruise_ship_info.csv")

df.head()


ParserError: ignored

In [3]:
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
#insurance_df = pd.read_sql_query('select * from medicalcosts',con=engine)

insurance_df = pd.DataFrame(engine.connect().execute(text('select * from medicalcosts')))

# No need for an open connection, because you're only doing a single query
engine.dispose()


insurance_df.head(10)

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.90
1,18,male,33.770,1,no,southeast,1725.55
2,28,male,33.000,3,no,southeast,4449.46
3,33,male,22.705,0,no,northwest,21984.50
4,32,male,28.880,0,no,northwest,3866.86
5,31,female,25.740,0,no,southeast,3756.62
6,46,female,33.440,1,no,southeast,8240.59
7,37,female,27.740,3,no,northwest,7281.51
8,37,male,29.830,2,no,northeast,6406.41
9,60,female,25.840,0,no,northwest,28923.10


In [4]:
insurance_df["is_male"] = pd.get_dummies(insurance_df.sex, drop_first=True)
insurance_df["is_smoker"] = pd.get_dummies(insurance_df.smoker, drop_first=True)

In [5]:
# Look at the values in the categorical feature
insurance_df['region'].value_counts()

southeast    364
southwest    325
northwest    325
northeast    324
Name: region, dtype: int64

In [6]:
insurance_df['charges'].min()

1121.87

In [7]:
insurance_df['charges'].max()

63770.4

In [8]:
regions_df = pd.get_dummies(insurance_df['region'])
regions_df.head()

,northeast,northwest,southeast,southwest
0,0,0,0,1
1,0,0,1,0
2,0,0,1,0
3,0,1,0,0
4,0,1,0,0


In [9]:
new_df = insurance_df.drop(['region'], axis =1).merge(regions_df,left_index=True, right_index=True)
new_df.head()

,age,sex,bmi,children,smoker,charges,is_male,is_smoker,northeast,northwest,southeast,southwest
0,19,female,27.900,0,yes,16884.90,0,1,0,0,0,1
1,18,male,33.770,1,no,1725.55,1,0,0,0,1,0
2,28,male,33.000,3,no,4449.46,1,0,0,0,1,0
3,33,male,22.705,0,no,21984.50,1,0,0,1,0,0
4,32,male,28.880,0,no,3866.86,1,0,0,1,0,0


In [10]:
# `Y` is the target variable
Y = insurance_df['charges']
# `X` is the feature set which includes the
# `is_male` and `is_smoker` variables
X = insurance_df[['is_male','is_smoker']]

In [11]:

# Create a `LinearRegression` model object
# from scikit-learn's linear_model module.
lrm = linear_model.LinearRegression()

# Fit method estimates the coefficients using OLS
lrm.fit(X, Y)

# Inspect the results
print('\nCoefficients: \n', lrm.coef_)
print('\nIntercept: \n', lrm.intercept_)


Coefficients: 
 [  -65.37868556 23622.13598049]

Intercept: 
 8466.035592512446


In [12]:
import statsmodels.api as sm

# You need to manually add a constant
# in statsmodels' sm
X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                charges   R-squared:                       0.620
Model:                            OLS   Adj. R-squared:                  0.619
Method:                 Least Squares   F-statistic:                     1088.
Date:                Tue, 02 Jan 2024   Prob (F-statistic):          4.78e-281
Time:                        19:02:56   Log-Likelihood:                -13831.
No. Observations:                1338   AIC:                         2.767e+04
Df Residuals:                    1335   BIC:                         2.768e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       8466.0356    303.541     27.891      0.000    7870.566    9061.506
is_male      -65.3787    409.809     -0.160      0.873    -869.318     738.561
is_smoker   2.362e+04    507.736     46.524      0.000    2.26e+04    2.46e+04
==============================================================================
Omnibus:                      135.842   Durbin-Watson:                   2.025
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              211.897
Skew:                           0.727   Prob(JB):                     9.71e-47
Kurtosis:                       4.299   Cond. No.                         2.97
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [15]:
from sklearn import neighbors
from sklearn.neighbors import KNeighborsRegressor
# Build the model
knn = KNeighborsRegressor(n_neighbors=10)
# `Y` is the target variable
Y = insurance_df['charges']
# `X` is the feature set which includes the
# `is_male` and `is_smoker` variables
X = insurance_df[['is_male','is_smoker']]
knn.fit(X, Y)

KNeighborsRegressor(n_neighbors=10)

In [39]:
from sklearn.model_selection import cross_val_score
score = cross_val_score(knn, X, Y, cv=5)
print("Unweighted Accuracy: %0.2f (+/- %0.2f)" % (score.mean(), score.std() * 2))


Unweighted Accuracy: 0.59 (+/- 0.08)
